In [2]:
import sys
sys.path=[x for x in sys.path if not "python2.7" in x]
import random
import os
import random
import numpy as np
import ujson as json
import matplotlib.pyplot as plt 
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms
import cv2

In [3]:
class LaneDataset(Dataset):
    I_H = 512
    I_W = 1024
    def __init__(self, data_path):
        if not os.path.exists(data_path):
            pths = ['/home/ubuntu/aiml/lane_model/']
            if not os.path.exists("data"):
                os.mkdir("data")
            with open("data/train.txt","w") as train, open("data/val.txt","w") as val, open("data/test.txt","w") as test:
                for pth in pths: # data/.txt에 모델 과정별로 입력될 사진자료 디렉토리 주소 입력
                    for i in range(1,51): # 1~50 까지 만들어진 폴더를 정해진 규칙에 따라 txt파일에 써줌
                        if i % 10 == 1:
                            val.write(pth+str(i)+"\n")
                        elif i % 10 == 2:
                            test.write(pth+str(i)+"\n")
                        else:
                            train.write(pth+str(i)+"\n")
                            
        with open(data_path) as txt:
            lines = txt.readlines()  # .txt파일 내용 읽어오기
        
        self.img_list = [] # .txt 경로를 따라가서 이미지 파일리스트를 저장함.
        for line in lines:
            self.img_list += [line.strip()+'/'+f for f in os.listdir(line.strip()) if ".jpg" in f or '.png' in f]
        # print([f for f in self.img_list if not os.path.exists(f.replace('.jpg','.json').replace('.png','.json'))])
        self.len = len(self.img_list)
        self.data_path = data_path

In [17]:
LaneDataset('/home/ubuntu/aiml/lane_model/data')

IsADirectoryError: [Errno 21] Is a directory: '/home/ubuntu/aiml/lane_model/data'

In [14]:
os.path.exists('/home/ubuntu/aiml/lane_model/data/val.txt')

True

In [5]:
pths = ['/home/ubuntu/aiml/lane_model/']
for pth in pths: 
    print(pth)

/home/ubuntu/aiml/lane_model/


In [6]:
data_path='/home/ubuntu/aiml/lane_model/data/val.txt'

with open(data_path) as txt:
            lines = txt.readlines()

print(lines)

['/home/ubuntu/aiml/data/traffic_lane/Validation/c_1280_720_daylight_validation_1/image/']


In [7]:
simg_list = []
for line in lines:
    simg_list += [line.strip()+'/'+f for f in os.listdir(line.strip()) if ".jpg" in f or '.png' in f]
# print([f for f in self.img_list if not os.path.exists(f.replace('.jpg','.json').replace('.png','.json'))])
slen = len(simg_list)
sdata_path = data_path

In [8]:
simg_list

['/home/ubuntu/aiml/data/traffic_lane/Validation/c_1280_720_daylight_validation_1/image//14089965.jpg',
 '/home/ubuntu/aiml/data/traffic_lane/Validation/c_1280_720_daylight_validation_1/image//14714322.jpg',
 '/home/ubuntu/aiml/data/traffic_lane/Validation/c_1280_720_daylight_validation_1/image//14916091.jpg',
 '/home/ubuntu/aiml/data/traffic_lane/Validation/c_1280_720_daylight_validation_1/image//9926985.jpg',
 '/home/ubuntu/aiml/data/traffic_lane/Validation/c_1280_720_daylight_validation_1/image//10429901.jpg',
 '/home/ubuntu/aiml/data/traffic_lane/Validation/c_1280_720_daylight_validation_1/image//16687042.jpg',
 '/home/ubuntu/aiml/data/traffic_lane/Validation/c_1280_720_daylight_validation_1/image//10430263.jpg',
 '/home/ubuntu/aiml/data/traffic_lane/Validation/c_1280_720_daylight_validation_1/image//4002804.jpg',
 '/home/ubuntu/aiml/data/traffic_lane/Validation/c_1280_720_daylight_validation_1/image//14385028.jpg',
 '/home/ubuntu/aiml/data/traffic_lane/Validation/c_1280_720_daylig

In [30]:
def make_gt_map(self, json_data, original_w, original_h):
        target_map = np.zeros((self.I_H, self.I_W),dtype=np.int32)
        annotation = json_data["annotations"]
        y_offset = original_h - original_w // 2
        for item in annotation:
            obj_class = item["class"]
            if obj_class == "traffic_lane":
                pos = item["data"]
                poly_points = np.array([([pt["x"]*self.I_W/original_w, (pt["y"] - y_offset)*self.I_H/(original_h-y_offset)]) for pt in pos]).astype(np.int32)
                cv2.polylines(target_map, [poly_points], False, 1,10)
            if obj_class == "stop_line":
                pos = item["data"]
                poly_points = np.array([([pt["x"]*self.I_W/original_w, (pt["y"] - y_offset)*self.I_H/(original_h-y_offset)]) for pt in pos]).astype(np.int32)
                cv2.polylines(target_map, [poly_points], False, 2,10)
            if obj_class == "crosswalk":
                pos = item["data"]
                poly_points = np.array([([pt["x"]*self.I_W/original_w, (pt["y"] - y_offset)*self.I_H/(original_h-y_offset)]) for pt in pos]).astype(np.int32)
                if len(poly_points) == 0:
                    continue
                cv2.fillPoly(target_map, [poly_points], 3)
        return target_map


I_H = 512
I_W = 1024
index=1
while True:
            img_path = simg_list[index]
            try:
                img = Image.open(img_path)
                break
            except:
                with open("error_files.txt",'a') as errlog:
                    errlog.write(img_path+'\n')
                    index = index + 1

w, h = img.size # 이미지 사이즈 조정
label_path = img_path.replace("image","json").replace('.jpg','.json').replace('.png','.json')
# 이미지 파일경로에 해당하는 부분을 json으로 바꿔서 json 파일이 열리게함
label_path 
with open(label_path) as json_file:
    json_data = json.load(json_file)
img_tensor = transforms.functional.to_tensor(transforms.functional.resized_crop(img, h-w//2, 0, w//2, w, (I_H,I_W)))
target_map = make_gt_map(json_data, w, h)

NameError: name 'make_gt_map' is not defined

In [7]:
with open('/home/ubuntu/aiml/lane_model/data/train.txt') as txt:
    lines = txt.readlines()  # .txt파일 내용 읽어오기

lines

['/home/ubuntu/aiml/data/traffic_lane/Training/c_1280_720_daylight_train_2/image/']

In [28]:
simg_list1 = [] # .txt 경로를 따라가서 이미지 파일리스트를 저장할 리스트 변수 생성.
simg_list = []
for line in lines:
    simg_list1 += [line.strip() + f for f in os.listdir(line.strip()) if ".jpg" in f or '.png' in f]
    #line.strip() = 공백제거
    # random.randrange(0, len(simg_list1))
for i in range(10) :
    index = random.randrange(0, len(simg_list1)-1)
    simg_list += [simg_list1[index]] # 대괄호로 안감싸주면  ㅂ,ㅈ,ㄷㄱ,ㅅ,ㅂ, 이렇게 저장됨
   

# print([f for f in self.img_list if not os.path.exists(f.replace('.jpg','.json').replace('.png','.json'))])

# print(os.listdir(line))
simg_list[]

IndexError: list index out of range

In [12]:
for line in lines:
    simg_list += [line.strip() + f for f in os.listdir(line.strip()) if ".jpg" in f or '.png' in f]
len(simg_list)
type(len(simg_list))
simg_list[len(simg_list)-1]



23542

In [11]:
listimg=[]
for i in range(3000) :
        listimg = simg_list[random.randrange(0, len(simg_list1))]
        print(listimg)

        

/home/ubuntu/aiml/data/traffic_lane/Training/c_1280_720_daylight_train_2/image/3804679.jpg
/home/ubuntu/aiml/data/traffic_lane/Training/c_1280_720_daylight_train_2/image/9864732.jpg
/home/ubuntu/aiml/data/traffic_lane/Training/c_1280_720_daylight_train_2/image/16785507.jpg
/home/ubuntu/aiml/data/traffic_lane/Training/c_1280_720_daylight_train_2/image/9922963.jpg
/home/ubuntu/aiml/data/traffic_lane/Training/c_1280_720_daylight_train_2/image/9923002.jpg
/home/ubuntu/aiml/data/traffic_lane/Training/c_1280_720_daylight_train_2/image/16778883.jpg
/home/ubuntu/aiml/data/traffic_lane/Training/c_1280_720_daylight_train_2/image/16785548.jpg
/home/ubuntu/aiml/data/traffic_lane/Training/c_1280_720_daylight_train_2/image/16800338.jpg
/home/ubuntu/aiml/data/traffic_lane/Training/c_1280_720_daylight_train_2/image/7012480.jpg
/home/ubuntu/aiml/data/traffic_lane/Training/c_1280_720_daylight_train_2/image/16812842.jpg
/home/ubuntu/aiml/data/traffic_lane/Training/c_1280_720_daylight_train_2/image/168043

In [38]:
slen = len(simg_list)
sdata_path = '/home/ubuntu/aiml/lane_model/data/train.txt'